<a href="https://colab.research.google.com/github/Swagat1342/Enhanced-Stock-Prediction-System-with-Risk-Management/blob/main/finance_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy pandas yfinance scikit-learn tensorflow plotly

In [ ]:
pip install alpha_vantage finnhub-python polygon-api-client

In [ ]:
pip install --upgrade yfinance


In [ ]:
pip install yfinance ta pandas numpy plotly


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=0ff8a5b6d84fa3fc27b63a3ab90995e519f8eceaf4019c6cacc832c8c4745937
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import yfinance as yf
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
INDIAN_STOCKS = {
    'NIFTY_50': {
        'RELIANCE.NS': 'Reliance Industries', 'TCS.NS': 'Tata Consultancy Services',
        'HDFCBANK.NS': 'HDFC Bank', 'INFY.NS': 'Infosys', 'ICICIBANK.NS': 'ICICI Bank',
        'HINDUNILVR.NS': 'Hindustan Unilever', 'ITC.NS': 'ITC Limited', 'SBIN.NS': 'State Bank of India',
        'BHARTIARTL.NS': 'Bharti Airtel', 'KOTAKBANK.NS': 'Kotak Mahindra Bank', 'LT.NS': 'Larsen & Toubro',
        'AXISBANK.NS': 'Axis Bank', 'BAJFINANCE.NS': 'Bajaj Finance', 'ASIANPAINT.NS': 'Asian Paints',
        'MARUTI.NS': 'Maruti Suzuki', 'HCLTECH.NS': 'HCL Technologies', 'WIPRO.NS': 'Wipro',
        'ULTRACEMCO.NS': 'UltraTech Cement', 'TITAN.NS': 'Titan Company', 'NESTLEIND.NS': 'Nestle India',
        'SUNPHARMA.NS': 'Sun Pharmaceutical', 'TECHM.NS': 'Tech Mahindra', 'POWERGRID.NS': 'Power Grid Corporation',
        'NTPC.NS': 'NTPC Limited', 'M&M.NS': 'Mahindra & Mahindra', 'TATAMOTORS.NS': 'Tata Motors',
        'BAJAJFINSV.NS': 'Bajaj Finserv', 'ADANIPORTS.NS': 'Adani Ports', 'COALINDIA.NS': 'Coal India',
        'ONGC.NS': 'ONGC', 'TATASTEEL.NS': 'Tata Steel', 'DIVISLAB.NS': 'Divi\'s Laboratories',
        'DRREDDY.NS': 'Dr. Reddy\'s Laboratories', 'GRASIM.NS': 'Grasim Industries', 'HINDALCO.NS': 'Hindalco Industries',
        'INDUSINDBK.NS': 'IndusInd Bank', 'JSWSTEEL.NS': 'JSW Steel', 'EICHERMOT.NS': 'Eicher Motors',
        'HEROMOTOCO.NS': 'Hero MotoCorp', 'CIPLA.NS': 'Cipla', 'BRITANNIA.NS': 'Britannia Industries',
        'SHREECEM.NS': 'Shree Cement', 'UPL.NS': 'UPL Limited', 'APOLLOHOSP.NS': 'Apollo Hospitals',
        'BPCL.NS': 'Bharat Petroleum', 'ADANIENT.NS': 'Adani Enterprises', 'BAJAJ-AUTO.NS': 'Bajaj Auto',
        'TATACONSUM.NS': 'Tata Consumer Products', 'SBILIFE.NS': 'SBI Life Insurance', 'HDFCLIFE.NS': 'HDFC Life Insurance',
    },
    'BANKING': {
        'HDFCBANK.NS': 'HDFC Bank', 'ICICIBANK.NS': 'ICICI Bank', 'SBIN.NS': 'State Bank of India',
        'KOTAKBANK.NS': 'Kotak Mahindra Bank', 'AXISBANK.NS': 'Axis Bank', 'INDUSINDBK.NS': 'IndusInd Bank',
        'BANDHANBNK.NS': 'Bandhan Bank', 'FEDERALBNK.NS': 'Federal Bank', 'IDFCFIRSTB.NS': 'IDFC First Bank',
        'PNB.NS': 'Punjab National Bank', 'BANKBARODA.NS': 'Bank of Baroda', 'CANBK.NS': 'Canara Bank',
    },
    'IT': {
        'TCS.NS': 'Tata Consultancy Services', 'INFY.NS': 'Infosys', 'HCLTECH.NS': 'HCL Technologies',
        'WIPRO.NS': 'Wipro', 'TECHM.NS': 'Tech Mahindra', 'LTIM.NS': 'LTIMindtree',
        'PERSISTENT.NS': 'Persistent Systems', 'COFORGE.NS': 'Coforge', 'MPHASIS.NS': 'Mphasis',
    },
    'PHARMA': {
        'SUNPHARMA.NS': 'Sun Pharmaceutical', 'DIVISLAB.NS': 'Divi\'s Laboratories', 'DRREDDY.NS': 'Dr. Reddy\'s Laboratories',
        'CIPLA.NS': 'Cipla', 'AUROPHARMA.NS': 'Aurobindo Pharma', 'LUPIN.NS': 'Lupin',
        'BIOCON.NS': 'Biocon', 'TORNTPHARM.NS': 'Torrent Pharmaceuticals', 'ALKEM.NS': 'Alkem Laboratories',
    },
    'AUTO': {
        'MARUTI.NS': 'Maruti Suzuki', 'TATAMOTORS.NS': 'Tata Motors', 'M&M.NS': 'Mahindra & Mahindra',
        'EICHERMOT.NS': 'Eicher Motors', 'HEROMOTOCO.NS': 'Hero MotoCorp', 'BAJAJ-AUTO.NS': 'Bajaj Auto',
        'TVSMOTOR.NS': 'TVS Motor', 'ASHOKLEY.NS': 'Ashok Leyland', 'MOTHERSON.NS': 'Motherson Sumi',
    },
    'FMCG': {
        'HINDUNILVR.NS': 'Hindustan Unilever', 'ITC.NS': 'ITC Limited', 'NESTLEIND.NS': 'Nestle India',
        'BRITANNIA.NS': 'Britannia Industries', 'DABUR.NS': 'Dabur India', 'MARICO.NS': 'Marico',
        'GODREJCP.NS': 'Godrej Consumer Products', 'COLPAL.NS': 'Colgate-Palmolive', 'TATACONSUM.NS': 'Tata Consumer Products',
    },
    'ENERGY': {
        'RELIANCE.NS': 'Reliance Industries', 'ONGC.NS': 'ONGC', 'BPCL.NS': 'Bharat Petroleum',
        'IOC.NS': 'Indian Oil Corporation', 'NTPC.NS': 'NTPC', 'POWERGRID.NS': 'Power Grid Corporation',
        'COALINDIA.NS': 'Coal India', 'GAIL.NS': 'GAIL India', 'ADANIGREEN.NS': 'Adani Green Energy',
        'TATAPOWER.NS': 'Tata Power',
    },
    'METALS': {
        'TATASTEEL.NS': 'Tata Steel', 'HINDALCO.NS': 'Hindalco Industries', 'JSWSTEEL.NS': 'JSW Steel',
        'COALINDIA.NS': 'Coal India', 'VEDL.NS': 'Vedanta', 'SAIL.NS': 'SAIL',
        'NMDC.NS': 'NMDC', 'HINDZINC.NS': 'Hindustan Zinc',
    },
    'INFRA': {
        'LT.NS': 'Larsen & Toubro', 'ADANIPORTS.NS': 'Adani Ports', 'ULTRACEMCO.NS': 'UltraTech Cement',
        'SHREECEM.NS': 'Shree Cement', 'GRASIM.NS': 'Grasim Industries', 'AMBUJACEM.NS': 'Ambuja Cements',
        'ACC.NS': 'ACC Limited',
    },
    'TELECOM': {
        'BHARTIARTL.NS': 'Bharti Airtel', 'IDEA.NS': 'Vodafone Idea',
    },
}

ALL_INDIAN_STOCKS = {}
for sector_stocks in INDIAN_STOCKS.values():
    ALL_INDIAN_STOCKS.update(sector_stocks)

print(f"✅ Loaded {len(ALL_INDIAN_STOCKS)} Indian stocks across {len(INDIAN_STOCKS)} sectors")


✅ Loaded 83 Indian stocks across 10 sectors


In [ ]:
def search_stock(query):
    """🔍 Search for stocks by name or ticker, or list sector stocks"""
    query = query.upper()

    # First check if it's a sector name
    if query in INDIAN_STOCKS:
        print(f"\n{'='*70}")
        print(f"📊 '{query}' is a SECTOR! Showing all {len(INDIAN_STOCKS[query])} stocks in this sector:")
        print(f"{'='*70}")
        print(f"{'Ticker':<20} {'Company Name'}")
        print("-" * 70)
        for ticker, name in INDIAN_STOCKS[query].items():
            print(f"{ticker:<20} {name}")
        print("="*70)
        return list(INDIAN_STOCKS[query].items())

    # Search for stocks by name or ticker
    results = []
    for ticker, name in ALL_INDIAN_STOCKS.items():
        if query in ticker.upper() or query in name.upper():
            results.append((ticker, name))

    if results:
        print(f"\n{'='*70}")
        print(f"🔍 SEARCH RESULTS FOR: '{query}' - Found {len(results)} matches")
        print(f"{'='*70}")
        print(f"{'Ticker':<20} {'Company Name'}")
        print("-" * 70)
        for ticker, name in results:
            print(f"{ticker:<20} {name}")
        print("="*70)
    else:
        print(f"\n❌ No stocks found matching '{query}'")
        print("\n💡 Tip: Try searching for:")
        print("   - Stock names: 'HDFC', 'Reliance', 'TCS', 'Infosys'")
        print("   - Keywords: 'Bank', 'Pharma', 'Tata', 'Auto'")
        print("   - Or sector names: 'BANKING', 'IT', 'PHARMA', 'AUTO', etc.")
        print("\n   Type 'sectors' to see all available sectors!")

    return results

def list_sectors():
    """📊 Display all available sectors"""
    print("\n" + "="*70)
    print("📊 AVAILABLE INDIAN STOCK SECTORS")
    print("="*70)
    for sector, stocks in INDIAN_STOCKS.items():
        print(f"  {sector:<15} - {len(stocks)} stocks")
    print(f"\n  Total: {len(ALL_INDIAN_STOCKS)} stocks across {len(INDIAN_STOCKS)} sectors")
    print("="*70)

def list_stocks_by_sector(sector):
    """📈 List all stocks in a specific sector"""
    sector = sector.upper()
    if sector in INDIAN_STOCKS:
        print(f"\n{'='*70}")
        print(f"📈 {sector} SECTOR - {len(INDIAN_STOCKS[sector])} STOCKS")
        print(f"{'='*70}")
        print(f"{'Ticker':<20} {'Company Name'}")
        print("-" * 70)
        for ticker, name in INDIAN_STOCKS[sector].items():
            print(f"{ticker:<20} {name}")
        print("="*70)
    else:
        print(f"❌ Sector '{sector}' not found.")
        list_sectors()

def get_stock_info(ticker):
    """Get basic stock information"""
    ticker = ticker.upper()
    if ticker in ALL_INDIAN_STOCKS:
        sector = next((s for s, stocks in INDIAN_STOCKS.items() if ticker in stocks), 'Unknown')
        return {'ticker': ticker, 'name': ALL_INDIAN_STOCKS[ticker], 'sector': sector}
    return None

def fetch_stock_data(ticker, period='1y', interval='1d'):
    """📥 Fetch stock data from Yahoo Finance"""
    print(f"\n🔹 Fetching data for {ticker}...")
    try:
        df = yf.download(ticker, period=period, interval=interval, progress=False)
        if df.empty:
            print("❌ No data returned")
            return None
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df.columns = [col.lower() for col in df.columns]
        print(f"✅ Fetched {len(df)} rows from {df.index[0].date()} to {df.index[-1].date()}")
        return df
    except Exception as e:
        print(f"❌ Error fetching data: {e}")
        return None

def add_technical_indicators(df):
    """📊 Add technical indicators to dataframe"""
    print("\n📊 Adding technical indicators...")
    df = df.copy()

    # Moving Averages
    for period in [20, 50, 200]:
        if len(df) >= period:
            df[f'sma_{period}'] = df['close'].rolling(window=period).mean()
            df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False).mean()

    # RSI
    if len(df) >= 14:
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / (loss + 1e-10)
        df['rsi_14'] = 100 - (100 / (1 + rs))

    # MACD
    exp1 = df['close'].ewm(span=12, adjust=False).mean()
    exp2 = df['close'].ewm(span=26, adjust=False).mean()
    df['macd'] = exp1 - exp2
    df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()

    # Bollinger Bands
    if len(df) >= 20:
        sma = df['close'].rolling(window=20).mean()
        std = df['close'].rolling(window=20).std()
        df['bb_upper'] = sma + (2 * std)
        df['bb_lower'] = sma - (2 * std)

    df.dropna(inplace=True)
    print(f"✅ Added indicators. Final dataset: {len(df)} rows")
    return df

def generate_signals(df):
    """🎯 Generate trading signals"""
    signals = {'timestamp': df.index[-1], 'close': df['close'].iloc[-1], 'signals': []}

    if 'rsi_14' in df.columns:
        rsi = df['rsi_14'].iloc[-1]
        if rsi < 30:
            signals['signals'].append(f"🟢 RSI ({rsi:.1f}): OVERSOLD - Buy signal")
        elif rsi > 70:
            signals['signals'].append(f"🔴 RSI ({rsi:.1f}): OVERBOUGHT - Sell signal")
        else:
            signals['signals'].append(f"⚪ RSI ({rsi:.1f}): NEUTRAL")

    if 'macd' in df.columns and 'macd_signal' in df.columns:
        if df['macd'].iloc[-1] > df['macd_signal'].iloc[-1]:
            signals['signals'].append(f"🟢 MACD: Bullish (above signal)")
        else:
            signals['signals'].append(f"🔴 MACD: Bearish (below signal)")

    return signals

def analyze_stock(ticker, period='1y', interval='1d', plot=True):
    """📈 Complete stock analysis"""
    ticker = ticker.upper()
    info = get_stock_info(ticker)

    print(f"\n{'='*70}")
    print(f"📈 ANALYZING: {ticker}")
    if info:
        print(f"Company: {info['name']}")
        print(f"Sector: {info['sector']}")
    print(f"{'='*70}")

    df = fetch_stock_data(ticker, period, interval)
    if df is None:
        return None

    df = add_technical_indicators(df)
    if df is None or len(df) == 0:
        return None

    # Statistics
    print(f"\n{'='*70}")
    print("📊 SUMMARY")
    print(f"{'='*70}")
    print(f"Current Price: ₹{df['close'].iloc[-1]:.2f}")
    print(f"High: ₹{df['high'].max():.2f}")
    print(f"Low: ₹{df['low'].min():.2f}")
    print(f"Avg Volume: {df['volume'].mean():,.0f}")

    # Signals
    signals = generate_signals(df)
    print(f"\n🎯 TRADING SIGNALS")
    print("-" * 70)
    for sig in signals['signals']:
        print(f"  {sig}")
    print("="*70)

    if plot:
        plot_chart(df, ticker)

    return df

def plot_chart(df, ticker):
    """📊 Plot interactive chart"""
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                        subplot_titles=[f'{ticker} - Price & Moving Averages', 'Volume', 'RSI'],
                        vertical_spacing=0.05, row_heights=[0.5, 0.25, 0.25])

    # Candlestick
    fig.add_trace(go.Candlestick(x=df.index, open=df['open'], high=df['high'],
                                  low=df['low'], close=df['close'], name='Price'), row=1, col=1)

    # Moving Averages
    for ma, color in [('sma_20', 'orange'), ('sma_50', 'blue'), ('sma_200', 'red')]:
        if ma in df.columns:
            fig.add_trace(go.Scatter(x=df.index, y=df[ma], mode='lines',
                                    name=ma.upper(), line=dict(color=color, width=1)), row=1, col=1)

    # Volume
    colors = ['green' if df['close'].iloc[i] >= df['open'].iloc[i] else 'red' for i in range(len(df))]
    fig.add_trace(go.Bar(x=df.index, y=df['volume'], name='Volume',
                        marker_color=colors, opacity=0.7), row=2, col=1)

    # RSI
    if 'rsi_14' in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df['rsi_14'], mode='lines',
                                name='RSI', line=dict(color='purple', width=2)), row=3, col=1)
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)

    fig.update_layout(title=f"{ticker} - Technical Analysis", template='plotly_dark',
                     height=900, showlegend=True, hovermode='x unified')
    fig.update_xaxes(rangeslider_visible=False)
    fig.show()


In [ ]:
print("\n" + "="*70)
print("🇮🇳 INDIAN STOCK MARKET ANALYSIS TOOL - READY!")
print("="*70)

print("\n📚 Available Functions:")
print("  search_stock('QUERY')              - 🔍 Search for stocks")
print("  list_sectors()                     - 📊 Show all sectors")
print("  list_stocks_by_sector('SECTOR')   - 📈 List stocks in sector")
print("  analyze_stock('TICKER')            - 📊 Full analysis with charts")

#Search
print("\n" + "="*70)
print("🔍 SEARCH FUNCTION")
print("="*70)
search_stock('TCS')

# Sectors
print("\n" + "="*70)
print(" 📊 SECTORS LIST")
print("="*70)
list_sectors()

# Interactive Search Feature
print("\n" + "="*70)
print("🔍 INTERACTIVE SEARCH")
print("="*70)
print("You can search for:")
print("  • Stock names: HDFC, Reliance, TCS, Infosys, Tata")
print("  • Keywords: Bank, Pharma, Auto, Cement")
print("  • Sectors: BANKING, IT, PHARMA, AUTO, ENERGY, FMCG")
print("  • Type 'sectors' to see all sectors")
print("  • Type 'quit' to exit")
print("="*70)

while True:
    try:
        user_input = input("\n🔍 Search: ").strip()

        if user_input.lower() in ['quit', 'exit', 'q']:
            print("\n✅ Exiting search. You can still use search_stock('QUERY') anytime!")
            break

        if user_input.lower() == 'sectors':
            list_sectors()
            continue

        if not user_input:
            print("⚠️ Please enter a search term!")
            continue

        results = search_stock(user_input)

        if results and len(results) > 0:
            # Ask if they want to analyze
            analyze_choice = input("\n💡 Analyze a stock? Enter ticker (or press Enter to continue): ").strip()
            if analyze_choice:
                analyze_stock(analyze_choice.upper(), period='6mo', plot=True)

    except KeyboardInterrupt:
        print("\n\n✅ Search interrupted. You can still use search_stock('QUERY') anytime!")
        break
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "="*70)
print("✅ TOOL READY FOR USE!")
print("="*70)
print("\n💡 You can still use these commands anytime:")
print("  search_stock('HDFC')")
print("  search_stock('BANK')")
print("  list_stocks_by_sector('IT')")
print("  analyze_stock('TCS.NS', period='6mo')")
print("  analyze_stock('RELIANCE.NS', period='1y')")
print("="*70)


🇮🇳 INDIAN STOCK MARKET ANALYSIS TOOL - READY!

📚 Available Functions:
  search_stock('QUERY')              - 🔍 Search for stocks
  list_sectors()                     - 📊 Show all sectors
  list_stocks_by_sector('SECTOR')   - 📈 List stocks in sector
  analyze_stock('TICKER')            - 📊 Full analysis with charts

🔍 SEARCH FUNCTION

🔍 SEARCH RESULTS FOR: 'TCS' - Found 1 matches
Ticker               Company Name
----------------------------------------------------------------------
TCS.NS               Tata Consultancy Services

 📊 SECTORS LIST

📊 AVAILABLE INDIAN STOCK SECTORS
  NIFTY_50        - 50 stocks
  BANKING         - 12 stocks
  IT              - 9 stocks
  PHARMA          - 9 stocks
  AUTO            - 9 stocks
  FMCG            - 9 stocks
  ENERGY          - 10 stocks
  METALS          - 8 stocks
  INFRA           - 7 stocks
  TELECOM         - 2 stocks

  Total: 83 stocks across 10 sectors

🔍 INTERACTIVE SEARCH
You can search for:
  • Stock names: HDFC, Reliance, TCS, Inf

##  DATA PREPARATION FOR **ML**

In [ ]:
def fetch_stock_data(ticker, period='5y'):
    """Fetch historical stock data"""
    try:
        print(f"Fetching data for {ticker}...")
        stock = yf.Ticker(ticker)
        df = stock.history(period=period)

        if df.empty:
            print(f"No data found for {ticker}")
            return None

        print(f"Data fetched: {len(df)} rows")
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def prepare_sequences(data, lookback):
    """Create sequences for LSTM training"""
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

def build_lstm_model(lookback, units=[50, 50], dropout=0.2):
    """Build LSTM model architecture"""
    model = Sequential()

    # First LSTM layer
    model.add(LSTM(units[0], return_sequences=True, input_shape=(lookback, 1)))
    model.add(Dropout(dropout))

    # Second LSTM layer
    model.add(LSTM(units[1], return_sequences=False))
    model.add(Dropout(dropout))

    # Output layer
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def train_stock_predictor(ticker, period='5y', lookback=50,
                         forecast_horizon=10, epochs=100, batch_size=32):
    """Complete training pipeline"""

    # Fetch and prepare data
    df = fetch_stock_data(ticker, period=period)
    if df is None:
        return None, None, None, None

    # Use closing prices
    close_prices = df['Close'].values.reshape(-1, 1)

    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(close_prices)

    # Split into train/test (80/20)
    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_size]
    test_data = scaled_data[train_size:]

    print(f"Train size: {len(train_data)}, Test size: {len(test_data)}")

    # Prepare sequences
    X_train, y_train = prepare_sequences(train_data, lookback)
    X_test, y_test = prepare_sequences(test_data, lookback)

    # Reshape for LSTM [samples, time steps, features]
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    print(f"Training sequences: {X_train.shape}, Test sequences: {X_test.shape}")

    # Build and train model
    print("\nBuilding LSTM model...")
    model = build_lstm_model(lookback)

    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    print("\nTraining model...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=1
    )

    # Make predictions on test set
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate metrics
    mse = np.mean((predictions - y_test_actual) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(predictions - y_test_actual))
    mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

    print(f"\n{'='*70}")
    print(f"MODEL PERFORMANCE METRICS")
    print(f"{'='*70}")
    print(f"RMSE: ${rmse:.2f}")
    print(f"MAE:  ${mae:.2f}")
    print(f"MAPE: {mape:.2f}%")

    return model, scaler, test_data, predictions, history, df

def forecast_future(model, scaler, last_sequence, forecast_horizon):
    """Generate future predictions"""
    future_predictions = []
    current_sequence = last_sequence.copy()

    for _ in range(forecast_horizon):
        # Reshape for prediction
        current_input = current_sequence.reshape(1, len(current_sequence), 1)

        # Predict next value
        next_pred = model.predict(current_input, verbose=0)
        future_predictions.append(next_pred[0, 0])

        # Update sequence
        current_sequence = np.append(current_sequence[1:], next_pred[0, 0])

    # Inverse transform predictions
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler.inverse_transform(future_predictions)

    return future_predictions

def complete_ml_forecast(ticker, period='5y', lookback=50,
                        forecast_horizon=10, epochs=100):
    """Complete ML forecasting pipeline with visualization"""

    print(f"\n{'='*70}")
    print(f"COMPLETE ML FORECASTING PIPELINE")
    print(f"{'='*70}")
    print(f"\n{'='*70}")
    print(f"TRAINING ML MODEL FOR {ticker}")
    print(f"{'='*70}\n")

    # Train model
    results = train_stock_predictor(
        ticker, period=period, lookback=lookback,
        forecast_horizon=forecast_horizon, epochs=epochs
    )

    if results[0] is None:
        return None, None, None, None

    model, scaler, test_data, predictions, history, df = results

    # Get last sequence for future forecasting
    last_sequence = test_data[-lookback:]

    # Generate future predictions
    print(f"\n{'='*70}")
    print(f"GENERATING {forecast_horizon}-DAY FORECAST")
    print(f"{'='*70}\n")

    future_pred = forecast_future(model, scaler, last_sequence, forecast_horizon)

    # Display forecast
    print(f"Future {forecast_horizon}-day predictions:")
    for i, pred in enumerate(future_pred, 1):
        print(f"Day {i}: ${pred[0]:.2f}")

    # Visualization with Plotly
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Model Training History', 'Test Set: Actual vs Predicted',
                       f'{ticker} - Historical + {forecast_horizon}-Day Forecast',
                       f'Recent 60 Days + Forecast Detail'),
        vertical_spacing=0.12,
        horizontal_spacing=0.10
    )

    # Plot 1: Training History
    fig.add_trace(
        go.Scatter(y=history.history['loss'], name='Train Loss',
                  mode='lines', line=dict(color='#00D9FF', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(y=history.history['val_loss'], name='Val Loss',
                  mode='lines', line=dict(color='#FF6B9D', width=2)),
        row=1, col=1
    )

    # Plot 2: Test Predictions vs Actual
    test_actual = scaler.inverse_transform(test_data[lookback:])
    fig.add_trace(
        go.Scatter(y=test_actual.flatten(), name='Actual',
                  mode='lines', line=dict(color='#00FF88', width=2)),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(y=predictions.flatten(), name='Predicted',
                  mode='lines', line=dict(color='#FFA500', width=2)),
        row=1, col=2
    )

    # Plot 3: Full Historical + Forecast
    historical_prices = df['Close'].values
    historical_dates = df.index

    fig.add_trace(
        go.Scatter(x=historical_dates, y=historical_prices, name='Historical',
                  mode='lines', line=dict(color='#00D9FF', width=2)),
        row=2, col=1
    )

    # Append future predictions
    future_dates = pd.date_range(start=historical_dates[-1], periods=forecast_horizon+1, freq='D')[1:]
    fig.add_trace(
        go.Scatter(x=future_dates, y=future_pred.flatten(), name='Forecast',
                  mode='lines+markers', line=dict(color='#FF1744', width=3, dash='dash'),
                  marker=dict(size=8, symbol='diamond')),
        row=2, col=1
    )

    # Plot 4:Forecast Detail
    recent_days = 60
    recent_prices = historical_prices[-recent_days:]
    recent_dates = historical_dates[-recent_days:]

    fig.add_trace(
        go.Scatter(x=recent_dates, y=recent_prices, name='Recent Historical',
                  mode='lines', line=dict(color='#00D9FF', width=2)),
        row=2, col=2
    )

    fig.add_trace(
        go.Scatter(x=future_dates, y=future_pred.flatten(), name='Forecast Detail',
                  mode='lines+markers', line=dict(color='#FF1744', width=3, dash='dash'),
                  marker=dict(size=8, symbol='diamond')),
        row=2, col=2
    )

    # Update layout with dark theme
    fig.update_layout(
        template='plotly_dark',
        height=900,
        showlegend=True,
        title_text=f"ML Stock Price Prediction - {ticker}",
        title_x=0.5,
        title_font=dict(size=20, color='#00D9FF'),
        hovermode='x unified'
    )

    # Update axes labels
    fig.update_xaxes(title_text="Epoch", row=1, col=1, gridcolor='#2D2D2D')
    fig.update_yaxes(title_text="Loss (MSE)", row=1, col=1, gridcolor='#2D2D2D')

    fig.update_xaxes(title_text="Time Steps", row=1, col=2, gridcolor='#2D2D2D')
    fig.update_yaxes(title_text="Price ($)", row=1, col=2, gridcolor='#2D2D2D')

    fig.update_xaxes(title_text="Date", row=2, col=1, gridcolor='#2D2D2D')
    fig.update_yaxes(title_text="Price ($)", row=2, col=1, gridcolor='#2D2D2D')

    fig.update_xaxes(title_text="Date", row=2, col=2, gridcolor='#2D2D2D')
    fig.update_yaxes(title_text="Price ($)", row=2, col=2, gridcolor='#2D2D2D')

    fig.show()

    return model, scaler, test_data, predictions


In [ ]:
model, scaler, test_data, predictions = complete_ml_forecast('TCS.NS', period='5y', lookback=50, forecast_horizon=10, epochs=100)


COMPLETE ML FORECASTING PIPELINE

TRAINING ML MODEL FOR TCS.NS

Fetching data for TCS.NS...
Data fetched: 1238 rows
Train size: 990, Test size: 248
Training sequences: (940, 50, 1), Test sequences: (198, 50, 1)

Building LSTM model...

Training model...
Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - loss: 0.0814 - mae: 0.2134 - val_loss: 0.0080 - val_mae: 0.0755
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.0080 - mae: 0.0707 - val_loss: 0.0031 - val_mae: 0.0425
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0057 - mae: 0.0577 - val_loss: 0.0028 - val_mae: 0.0411
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0047 - mae: 0.0532 - val_loss: 0.0024 - val_mae: 0.0384
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - loss: 0.0038 - mae: 0.0473 - val_loss: 0.0023 - val_mae: 0.0376
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0041 - mae: 0.0489 - val_loss: 0.0025 - val_mae: 0.0381
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━

In [ ]:

# ==================== LSTM ARCHITECTURES ====================

class AdvancedLSTMArchitectures:
    """5 Advanced LSTM Architectures"""

    @staticmethod
    def bidirectional_lstm(lookback, n_features=1, units=[128, 64], dropout=0.3):
        model = keras.Sequential([
            Input(shape=(lookback, n_features)),
            Bidirectional(LSTM(units[0], return_sequences=True)),
            Dropout(dropout),
            LayerNormalization(),
            Bidirectional(LSTM(units[1], return_sequences=False)),
            Dropout(dropout),
            LayerNormalization(),
            Dense(64, activation='relu'),
            Dropout(dropout),
            Dense(32, activation='relu'),
            Dense(1)
        ], name='BiLSTM')
        model.compile(optimizer=Adam(learning_rate=0.001), loss='huber', metrics=['mae'])
        return model

    @staticmethod
    def gru_model(lookback, n_features=1, units=[128, 64], dropout=0.3):
        model = keras.Sequential([
            Input(shape=(lookback, n_features)),
            GRU(units[0], return_sequences=True),
            Dropout(dropout),
            LayerNormalization(),
            GRU(units[1], return_sequences=False),
            Dropout(dropout),
            LayerNormalization(),
            Dense(64, activation='relu'),
            Dropout(dropout),
            Dense(32, activation='relu'),
            Dense(1)
        ], name='GRU')
        model.compile(optimizer=Adam(learning_rate=0.001), loss='huber', metrics=['mae'])
        return model

    @staticmethod
    def attention_lstm(lookback, n_features=1, units=[128, 64], dropout=0.3):
        inputs = Input(shape=(lookback, n_features))
        lstm_out = LSTM(units[0], return_sequences=True)(inputs)
        lstm_out = Dropout(dropout)(lstm_out)
        lstm_out = LayerNormalization()(lstm_out)
        attention = Attention()([lstm_out, lstm_out])
        attention = LayerNormalization()(attention)
        lstm_out2 = LSTM(units[1], return_sequences=False)(attention)
        lstm_out2 = Dropout(dropout)(lstm_out2)
        lstm_out2 = LayerNormalization()(lstm_out2)
        dense = Dense(64, activation='relu')(lstm_out2)
        dense = Dropout(dropout)(dense)
        dense = Dense(32, activation='relu')(dense)
        outputs = Dense(1)(dense)
        model = Model(inputs=inputs, outputs=outputs, name='AttentionLSTM')
        model.compile(optimizer=Adam(learning_rate=0.001), loss='huber', metrics=['mae'])
        return model

    @staticmethod
    def stacked_lstm(lookback, n_features=1, units=[128, 96, 64], dropout=0.3):
        model = keras.Sequential([
            Input(shape=(lookback, n_features)),
            LSTM(units[0], return_sequences=True),
            Dropout(dropout),
            LayerNormalization(),
            LSTM(units[1], return_sequences=True),
            Dropout(dropout),
            LayerNormalization(),
            LSTM(units[2], return_sequences=False),
            Dropout(dropout),
            LayerNormalization(),
            Dense(64, activation='relu'),
            Dropout(dropout),
            Dense(32, activation='relu'),
            Dense(1)
        ], name='StackedLSTM')
        model.compile(optimizer=Adam(learning_rate=0.001), loss='huber', metrics=['mae'])
        return model

    @staticmethod
    def cnn_lstm(lookback, n_features=1, dropout=0.3):
        model = keras.Sequential([
            Input(shape=(lookback, n_features)),
            Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
            MaxPooling1D(pool_size=2),
            Dropout(dropout),
            Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
            MaxPooling1D(pool_size=2),
            Dropout(dropout),
            LSTM(64, return_sequences=False),
            Dropout(dropout),
            Dense(32, activation='relu'),
            Dense(1)
        ], name='CNN_LSTM')
        model.compile(optimizer=Adam(learning_rate=0.001), loss='huber', metrics=['mae'])
        return model


# ==================== FEATURE ENGINEERING ====================

class FeatureEngineer:
    """Feature engineering - 60+ technical indicators"""

    @staticmethod
    def add_all_features(df):
        df = df.copy()

        # Fix multi-level columns
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        # Basic features
        df['returns'] = df['Close'].pct_change()
        df['log_returns'] = np.log(df['Close'] / df['Close'].shift(1))

        # Moving averages
        for period in [5, 10, 20, 50, 100, 200]:
            df[f'sma_{period}'] = df['Close'].rolling(window=period).mean()
            df[f'ema_{period}'] = df['Close'].ewm(span=period, adjust=False).mean()

        df['price_to_sma20'] = df['Close'] / df['sma_20']
        df['price_to_sma50'] = df['Close'] / df['sma_50']

        # Volatility
        df['volatility_10'] = df['returns'].rolling(window=10).std()
        df['volatility_30'] = df['returns'].rolling(window=30).std()
        df['volatility_ratio'] = df['volatility_10'] / df['volatility_30']

        # RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['rsi'] = 100 - (100 / (1 + rs))

        # MACD
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['macd'] = exp1 - exp2
        df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
        df['macd_hist'] = df['macd'] - df['macd_signal']

        # Bollinger Bands
        sma_20 = df['Close'].rolling(window=20).mean()
        std_20 = df['Close'].rolling(window=20).std()
        df['bb_upper'] = sma_20 + (2 * std_20)
        df['bb_lower'] = sma_20 - (2 * std_20)
        df['bb_width'] = ((df['bb_upper'] - df['bb_lower']) / sma_20).values
        df['bb_position'] = (df['Close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])

        # Volume
        df['volume_sma_20'] = df['Volume'].rolling(window=20).mean()
        df['volume_ratio'] = (df['Volume'] / df['volume_sma_20']).values
        df['obv'] = (np.sign(df['returns']) * df['Volume']).fillna(0).cumsum()

        # Momentum
        for period in [5, 10, 20]:
            df[f'momentum_{period}'] = df['Close'] - df['Close'].shift(period)
            df[f'roc_{period}'] = ((df['Close'] - df['Close'].shift(period)) /
                                    df['Close'].shift(period)) * 100

        # ATR
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = np.max(ranges, axis=1)
        df['atr'] = true_range.rolling(14).mean()
        df['atr_percent'] = (df['atr'] / df['Close']) * 100

        # Stochastic
        low_14 = df['Low'].rolling(window=14).min()
        high_14 = df['High'].rolling(window=14).max()
        df['stoch_k'] = 100 * ((df['Close'] - low_14) / (high_14 - low_14))
        df['stoch_d'] = df['stoch_k'].rolling(window=3).mean()

        # Market index correlation
        try:
            index_data = yf.download('^NSEI', start=df.index[0], end=df.index[-1], progress=False)
            if isinstance(index_data.columns, pd.MultiIndex):
                index_data.columns = index_data.columns.get_level_values(0)
            df['index_close'] = index_data['Close']
            df['index_returns'] = df['index_close'].pct_change()
            df['correlation_30'] = df['returns'].rolling(window=30).corr(df['index_returns'])
        except:
            df['correlation_30'] = 0

        return df


# ==================== COMPLETE PIPELINE ====================

def run_complete_analysis(ticker='TCS.NS', period='5y', lookback=60,
                         epochs=100, days_forecast=30):
    """
    🚀 ONE-CLICK COMPLETE ANALYSIS

    Args:
        ticker: Stock symbol (e.g., 'TCS.NS')
        period: Data period ('1y', '2y', '5y', '10y')
        lookback: Days to look back (default: 60)
        epochs: Training epochs (default: 100)
        days_forecast: Days to forecast (default: 30)

    Returns:
        Dictionary with all results
    """

    print("\n" + "🚀"*40)
    print("COMPLETE STOCK PREDICTION PIPELINE")
    print("🚀"*40)
    print(f"\nTicker: {ticker} | Period: {period} | Forecast: {days_forecast} days\n")

    # Step 1: Download data
    print("📥 STEP 1/6: Downloading data...")
    data = yf.download(ticker, period=period, progress=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    print(f"✅ Downloaded {len(data)} rows\n")

    # Step 2: Engineer features
    print("🔧 STEP 2/6: Engineering features...")
    engineer = FeatureEngineer()
    data = engineer.add_all_features(data)
    data.dropna(inplace=True)
    print(f"✅ {len(data.columns)} features created\n")

    # Step 3: Prepare sequences
    print("📊 STEP 3/6: Preparing sequences...")
    feature_cols = [
        'Close', 'Volume', 'returns', 'volatility_10',
        'rsi', 'macd', 'bb_position', 'volume_ratio',
        'sma_20', 'ema_50', 'atr_percent', 'correlation_30'
    ]

    scaler = MinMaxScaler()
    features = data[feature_cols].values
    scaled = scaler.fit_transform(features)

    X, y = [], []
    for i in range(lookback, len(scaled)):
        X.append(scaled[i-lookback:i])
        y.append(scaled[i, 0])

    X, y = np.array(X), np.array(y)

    train_size = int(len(X) * 0.7)
    val_size = int(len(X) * 0.15)

    X_train, X_val, X_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
    y_train, y_val, y_test = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]

    print(f"✅ Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}\n")

    # Step 4: Train models
    print("🤖 STEP 4/6: Training models...")
    architectures = AdvancedLSTMArchitectures()
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=0),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6, verbose=0)
    ]

    models = {}
    n_features = X_train.shape[2]

    model_list = [
        ('BiLSTM', architectures.bidirectional_lstm),
        ('GRU', architectures.gru_model),
        ('AttentionLSTM', architectures.attention_lstm),
        ('StackedLSTM', architectures.stacked_lstm),
        ('CNN_LSTM', architectures.cnn_lstm)
    ]

    for name, model_func in model_list:
        print(f"  Training {name}...", end=' ')
        model = model_func(lookback, n_features)
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                 epochs=epochs, batch_size=32, callbacks=callbacks, verbose=0)
        models[name] = model
        print("✅")

    print(f"✅ {len(models)} models trained\n")

    # Step 5: Evaluate
    print("📈 STEP 5/6: Evaluating models...")
    results = {}
    predictions = {}

    def inverse_transform(pred_scaled):
        pred_full = np.zeros((len(pred_scaled), len(feature_cols)))
        pred_full[:, 0] = pred_scaled
        return scaler.inverse_transform(pred_full)[:, 0]

    y_actual = inverse_transform(y_test)

    for name, model in models.items():
        pred_scaled = model.predict(X_test, verbose=0).flatten()
        pred = inverse_transform(pred_scaled)

        rmse = np.sqrt(mean_squared_error(y_actual, pred))
        mae = mean_absolute_error(y_actual, pred)
        mape = np.mean(np.abs((y_actual - pred) / y_actual)) * 100
        r2 = r2_score(y_actual, pred)

        results[name] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R2': r2}
        predictions[name] = pred

        print(f"  {name:15s}: RMSE=₹{rmse:7.2f}, MAPE={mape:5.2f}%, R²={r2:.4f}")

    best_model = min(results.items(), key=lambda x: x[1]['RMSE'])
    print(f"\n🏆 Best: {best_model[0]} (RMSE: ₹{best_model[1]['RMSE']:.2f})\n")

    # Step 6: Future predictions
    print("🔮 STEP 6/6: Predicting future...")
    last_sequence = scaled[-lookback:]
    future_predictions = {name: [] for name in models.keys()}

    for day in range(days_forecast):
        current_seq = last_sequence[-lookback:].reshape(1, lookback, n_features)
        day_preds = []
        for name, model in models.items():
            pred = model.predict(current_seq, verbose=0)[0, 0]
            future_predictions[name].append(pred)
            day_preds.append(pred)

        new_row = last_sequence[-1].copy()
        new_row[0] = np.mean(day_preds)
        last_sequence = np.vstack([last_sequence, new_row])

    future_dates = pd.date_range(start=data.index[-1] + timedelta(days=1),
                                  periods=days_forecast, freq='D')
    future_results = {}
    for name, preds in future_predictions.items():
        future_results[name] = inverse_transform(np.array(preds))

    print(f"✅ {days_forecast} days predicted\n")

    # Visualization
    print("📊 Creating visualizations...")
    test_start_idx = len(data) - len(y_actual)
    test_dates = data.index[test_start_idx:]

    # Prediction comparison
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=test_dates, y=y_actual, mode='lines',
                              name='Actual', line=dict(color='black', width=3)))
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    for i, (name, pred) in enumerate(predictions.items()):
        fig1.add_trace(go.Scatter(x=test_dates, y=pred, mode='lines', name=name,
                                  line=dict(color=colors[i], width=2, dash='dash')))
    fig1.update_layout(title=f'{ticker} - Model Predictions', xaxis_title='Date',
                      yaxis_title='Price (₹)', height=500, template='plotly_white')
    fig1.show()

    # Future forecast
    fig2 = go.Figure()
    historical = data['Close'].values[-60:]
    historical_dates = data.index[-60:]
    fig2.add_trace(go.Scatter(x=historical_dates, y=historical, mode='lines',
                             name='Historical', line=dict(color='black', width=2)))
    for i, (name, pred) in enumerate(future_results.items()):
        fig2.add_trace(go.Scatter(x=future_dates, y=pred, mode='lines',
                                 name=f'{name} Forecast',
                                 line=dict(color=colors[i], width=2, dash='dash')))
    fig2.add_shape(type="line", x0=data.index[-1], x1=data.index[-1],
                  y0=0, y1=1, yref="paper", line=dict(color="gray", width=2, dash="dot"))
    fig2.update_layout(title=f'{ticker} - {days_forecast}-Day Forecast',
                      xaxis_title='Date', yaxis_title='Price (₹)',
                      height=500, template='plotly_white')
    fig2.show()

    # Summary
    current_price = data['Close'].values[-1]
    predicted_price = np.mean([p[-1] for p in future_results.values()])

    print("\n" + "🎉"*40)
    print("ANALYSIS COMPLETE!")
    print("🎉"*40)
    print(f"\n📊 SUMMARY FOR {ticker}")
    print("="*80)
    print(f"Current Price:     ₹{current_price:.2f}")
    print(f"Predicted ({days_forecast}d):   ₹{predicted_price:.2f}")
    print(f"Expected Change:   {((predicted_price/current_price)-1)*100:+.2f}%")
    print(f"Best Model:        {best_model[0]} (MAPE: {best_model[1]['MAPE']:.2f}%)")
    print("="*80 + "\n")

    return {
        'ticker': ticker,
        'data': data,
        'models': models,
        'results': results,
        'predictions': predictions,
        'y_actual': y_actual,
        'future_predictions': future_results,
        'future_dates': future_dates,
        'current_price': current_price,
        'predicted_price': predicted_price,
        'best_model': best_model[0]
    }


# ==================== USAGE EXAMPLES ====================

if __name__ == "__main__":
    print("\n" + "="*80)
    print("🎯 STOCK PREDICTION SYSTEM READY!")
    print("="*80)
    print("\n📌 USAGE:")
    print("  results = run_complete_analysis('TCS.NS')")
    print("\n📌 EXAMPLES:")
    print("  run_complete_analysis('INFY.NS', period='3y', days_forecast=60)")
    print("  run_complete_analysis('RELIANCE.NS', epochs=150)")
    print("\n" + "="*80 + "\n")

    # Uncomment to run
    # results = run_complete_analysis('TCS.NS', period='2y', epochs=50)


🎯 STOCK PREDICTION SYSTEM READY!

📌 USAGE:
  results = run_complete_analysis('TCS.NS')

📌 EXAMPLES:
  run_complete_analysis('INFY.NS', period='3y', days_forecast=60)
  run_complete_analysis('RELIANCE.NS', epochs=150)




In [1]:

def fetch_stock_data(ticker, period='5y'):
    """Fetch historical stock data"""
    try:
        print(f"Fetching data for {ticker}...")
        stock = yf.Ticker(ticker)
        df = stock.history(period=period)

        if df.empty:
            print(f"No data found for {ticker}")
            return None

        print(f"Data fetched: {len(df)} rows")
        return df
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def prepare_sequences(data, lookback):
    """Create sequences for LSTM training"""
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

def build_lstm_model(lookback, units=[50, 50], dropout=0.2):
    """Build LSTM model architecture"""
    model = Sequential()

    # First LSTM layer
    model.add(LSTM(units[0], return_sequences=True, input_shape=(lookback, 1)))
    model.add(Dropout(dropout))

    # Second LSTM layer
    model.add(LSTM(units[1], return_sequences=False))
    model.add(Dropout(dropout))

    # Output layer
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

def train_stock_predictor(ticker, period='5y', lookback=50,
                         forecast_horizon=10, epochs=100, batch_size=32):
    """Complete training pipeline"""

    # Fetch and prepare data
    df = fetch_stock_data(ticker, period=period)
    if df is None:
        return None, None, None, None

    # Use closing prices
    close_prices = df['Close'].values.reshape(-1, 1)

    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(close_prices)

    # Split into train/test (80/20)
    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_size]
    test_data = scaled_data[train_size:]

    print(f"Train size: {len(train_data)}, Test size: {len(test_data)}")

    # Prepare sequences
    X_train, y_train = prepare_sequences(train_data, lookback)
    X_test, y_test = prepare_sequences(test_data, lookback)

    # Reshape for LSTM [samples, time steps, features]
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    print(f"Training sequences: {X_train.shape}, Test sequences: {X_test.shape}")

    # Build and train model
    print("\nBuilding LSTM model...")
    model = build_lstm_model(lookback)

    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    print("\nTraining model...")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=1
    )

    # Make predictions on test set
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate metrics
    mse = np.mean((predictions - y_test_actual) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(predictions - y_test_actual))
    mape = np.mean(np.abs((y_test_actual - predictions) / y_test_actual)) * 100

    print(f"\n{'='*70}")
    print(f"MODEL PERFORMANCE METRICS")
    print(f"{'='*70}")
    print(f"RMSE: ${rmse:.2f}")
    print(f"MAE:  ${mae:.2f}")
    print(f"MAPE: {mape:.2f}%")

    return model, scaler, test_data, predictions, history, df

def forecast_future(model, scaler, last_sequence, forecast_horizon):
    """Generate future predictions"""
    future_predictions = []
    current_sequence = last_sequence.copy()

    for _ in range(forecast_horizon):
        # Reshape for prediction
        current_input = current_sequence.reshape(1, len(current_sequence), 1)

        # Predict next value
        next_pred = model.predict(current_input, verbose=0)
        future_predictions.append(next_pred[0, 0])

        # Update sequence
        current_sequence = np.append(current_sequence[1:], next_pred[0, 0])

    # Inverse transform predictions
    future_predictions = np.array(future_predictions).reshape(-1, 1)
    future_predictions = scaler.inverse_transform(future_predictions)

    return future_predictions

def complete_ml_forecast(ticker, period='5y', lookback=50,
                        forecast_horizon=10, epochs=100):
    """Complete ML forecasting pipeline with visualization"""

    print(f"\n{'='*70}")
    print(f"COMPLETE ML FORECASTING PIPELINE")
    print(f"{'='*70}")
    print(f"\n{'='*70}")
    print(f"TRAINING ML MODEL FOR {ticker}")
    print(f"{'='*70}\n")

    # Train model
    results = train_stock_predictor(
        ticker, period=period, lookback=lookback,
        forecast_horizon=forecast_horizon, epochs=epochs
    )

    if results[0] is None:
        return None, None, None, None

    model, scaler, test_data, predictions, history, df = results

    # Get last sequence for future forecasting
    last_sequence = test_data[-lookback:]

    # Generate future predictions
    print(f"\n{'='*70}")
    print(f"GENERATING {forecast_horizon}-DAY FORECAST")
    print(f"{'='*70}\n")

    future_pred = forecast_future(model, scaler, last_sequence, forecast_horizon)

    # Display forecast
    print(f"Future {forecast_horizon}-day predictions:")
    for i, pred in enumerate(future_pred, 1):
        print(f"Day {i}: ${pred[0]:.2f}")

    # Visualization with Plotly
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Model Training History', 'Test Set: Actual vs Predicted',
                       f'{ticker} - Historical + {forecast_horizon}-Day Forecast',
                       f'Recent 60 Days + Forecast Detail'),
        vertical_spacing=0.12,
        horizontal_spacing=0.10
    )

    # Plot 1: Training History
    fig.add_trace(
        go.Scatter(y=history.history['loss'], name='Train Loss',
                  mode='lines', line=dict(color='#00D9FF', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(y=history.history['val_loss'], name='Val Loss',
                  mode='lines', line=dict(color='#FF6B9D', width=2)),
        row=1, col=1
    )

    # Plot 2: Test Predictions vs Actual
    test_actual = scaler.inverse_transform(test_data[lookback:])
    fig.add_trace(
        go.Scatter(y=test_actual.flatten(), name='Actual',
                  mode='lines', line=dict(color='#00FF88', width=2)),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(y=predictions.flatten(), name='Predicted',
                  mode='lines', line=dict(color='#FFA500', width=2)),
        row=1, col=2
    )

    # Plot 3: Full Historical + Forecast
    historical_prices = df['Close'].values
    historical_dates = df.index

    fig.add_trace(
        go.Scatter(x=historical_dates, y=historical_prices, name='Historical',
                  mode='lines', line=dict(color='#00D9FF', width=2)),
        row=2, col=1
    )

    # Append future predictions
    future_dates = pd.date_range(start=historical_dates[-1], periods=forecast_horizon+1, freq='D')[1:]
    fig.add_trace(
        go.Scatter(x=future_dates, y=future_pred.flatten(), name='Forecast',
                  mode='lines+markers', line=dict(color='#FF1744', width=3, dash='dash'),
                  marker=dict(size=8, symbol='diamond')),
        row=2, col=1
    )

    # Plot 4: Recent + Forecast Detail
    recent_days = 60
    recent_prices = historical_prices[-recent_days:]
    recent_dates = historical_dates[-recent_days:]

    fig.add_trace(
        go.Scatter(x=recent_dates, y=recent_prices, name='Recent Historical',
                  mode='lines', line=dict(color='#00D9FF', width=2)),
        row=2, col=2
    )

    fig.add_trace(
        go.Scatter(x=future_dates, y=future_pred.flatten(), name='Forecast Detail',
                  mode='lines+markers', line=dict(color='#FF1744', width=3, dash='dash'),
                  marker=dict(size=8, symbol='diamond')),
        row=2, col=2
    )

    # Update layout with dark theme
    fig.update_layout(
        template='plotly_dark',
        height=900,
        showlegend=True,
        title_text=f"ML Stock Price Prediction - {ticker}",
        title_x=0.5,
        title_font=dict(size=20, color='#00D9FF'),
        hovermode='x unified'
    )

    # Update axes labels
    fig.update_xaxes(title_text="Epoch", row=1, col=1, gridcolor='#2D2D2D')
    fig.update_yaxes(title_text="Loss (MSE)", row=1, col=1, gridcolor='#2D2D2D')

    fig.update_xaxes(title_text="Time Steps", row=1, col=2, gridcolor='#2D2D2D')
    fig.update_yaxes(title_text="Price ($)", row=1, col=2, gridcolor='#2D2D2D')

    fig.update_xaxes(title_text="Date", row=2, col=1, gridcolor='#2D2D2D')
    fig.update_yaxes(title_text="Price ($)", row=2, col=1, gridcolor='#2D2D2D')

    fig.update_xaxes(title_text="Date", row=2, col=2, gridcolor='#2D2D2D')
    fig.update_yaxes(title_text="Price ($)", row=2, col=2, gridcolor='#2D2D2D')

    fig.show()

    return model, scaler, test_data, predictions

# To use, simply call:
# model, scaler, test_data, predictions = complete_ml_forecast('M&M.NS', period='5y', lookback=50, forecast_horizon=10, epochs=100)

In [3]:
model, scaler, test_data, predictions = complete_ml_forecast('TCS.NS', period='5y', lookback=50, forecast_horizon=10, epochs=100)


COMPLETE ML FORECASTING PIPELINE

TRAINING ML MODEL FOR TCS.NS

Fetching data for TCS.NS...
Data fetched: 1237 rows
Train size: 989, Test size: 248
Training sequences: (939, 50, 1), Test sequences: (198, 50, 1)

Building LSTM model...

Training model...
Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step - loss: 0.0501 - mae: 0.1714 - val_loss: 0.0044 - val_mae: 0.0518
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 0.0063 - mae: 0.0618 - val_loss: 0.0030 - val_mae: 0.0408
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0043 - mae: 0.0520 - val_loss: 0.0038 - val_mae: 0.0462
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.0042 - mae: 0.0499 - val_loss: 0.0022 - val_mae: 0.0362
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - loss: 0.0040 - mae: 0.0496 - val_loss: 0.0021 - val_mae: 0.0363
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 0.0041 - mae: 0.0504 - val_loss: 0.0021 - val_mae: 0.0367
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━

In [12]:


# ==================== FEATURE ENGINEERING ====================

class FeatureEngineer:
    """Feature engineering - technical indicators"""

    @staticmethod
    def add_all_features(df):
        df = df.copy()

        # Fix multi-level columns
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        # Basic features
        df['returns'] = df['Close'].pct_change()
        df['log_returns'] = np.log(df['Close'] / df['Close'].shift(1))

        # Moving averages
        for period in [5, 10, 20, 50, 100, 200]:
            df[f'sma_{period}'] = df['Close'].rolling(window=period).mean()
            df[f'ema_{period}'] = df['Close'].ewm(span=period, adjust=False).mean()

        df['price_to_sma20'] = df['Close'] / df['sma_20']
        df['price_to_sma50'] = df['Close'] / df['sma_50']

        # Volatility
        df['volatility_10'] = df['returns'].rolling(window=10).std()
        df['volatility_30'] = df['returns'].rolling(window=30).std()
        df['volatility_ratio'] = df['volatility_10'] / (df['volatility_30'] + 1e-10)

        # RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / (loss + 1e-10)
        df['rsi'] = 100 - (100 / (1 + rs))

        # MACD
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['macd'] = exp1 - exp2
        df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
        df['macd_hist'] = df['macd'] - df['macd_signal']

        # Bollinger Bands
        sma_20 = df['Close'].rolling(window=20).mean()
        std_20 = df['Close'].rolling(window=20).std()
        df['bb_upper'] = sma_20 + (2 * std_20)
        df['bb_lower'] = sma_20 - (2 * std_20)
        df['bb_width'] = ((df['bb_upper'] - df['bb_lower']) / (sma_20 + 1e-10)).values
        df['bb_position'] = (df['Close'] - df['bb_lower']) / ((df['bb_upper'] - df['bb_lower']) + 1e-10)

        # Volume
        df['volume_sma_20'] = df['Volume'].rolling(window=20).mean()
        df['volume_ratio'] = (df['Volume'] / (df['volume_sma_20'] + 1e-10)).values
        df['obv'] = (np.sign(df['returns']) * df['Volume']).fillna(0).cumsum()

        # Momentum
        for period in [5, 10, 20]:
            df[f'momentum_{period}'] = df['Close'] - df['Close'].shift(period)
            df[f'roc_{period}'] = ((df['Close'] - df['Close'].shift(period)) /
                                    (df['Close'].shift(period) + 1e-10)) * 100

        # ATR
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = np.max(ranges, axis=1)
        df['atr'] = true_range.rolling(14).mean()
        df['atr_percent'] = (df['atr'] / (df['Close'] + 1e-10)) * 100

        # Stochastic
        low_14 = df['Low'].rolling(window=14).min()
        high_14 = df['High'].rolling(window=14).max()
        df['stoch_k'] = 100 * ((df['Close'] - low_14) / ((high_14 - low_14) + 1e-10))
        df['stoch_d'] = df['stoch_k'].rolling(window=3).mean()

        return df


# ==================== BACKTESTING ENGINE ====================

class BacktestEngine:
    """Core backtesting engine"""

    def __init__(self, initial_capital=100000, transaction_cost=0.001, slippage=0.0005):
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        self.slippage = slippage
        self.reset()

    def reset(self):
        self.capital = self.initial_capital
        self.positions = 0
        self.trades = []
        self.portfolio_values = []
        self.dates = []
        self.holdings = []

    def buy(self, date, price, quantity=None):
        if quantity is None:
            quantity = int(self.capital / (price * (1 + self.transaction_cost + self.slippage)))

        cost = quantity * price * (1 + self.transaction_cost + self.slippage)

        if cost <= self.capital and quantity > 0:
            self.capital -= cost
            self.positions += quantity
            self.trades.append({
                'date': date,
                'type': 'BUY',
                'price': price,
                'quantity': quantity,
                'cost': cost,
                'capital': self.capital,
                'positions': self.positions
            })
            return True
        return False

    def sell(self, date, price, quantity=None):
        if quantity is None:
            quantity = self.positions

        if quantity <= self.positions and quantity > 0:
            revenue = quantity * price * (1 - self.transaction_cost - self.slippage)
            self.capital += revenue
            self.positions -= quantity
            self.trades.append({
                'date': date,
                'type': 'SELL',
                'price': price,
                'quantity': quantity,
                'revenue': revenue,
                'capital': self.capital,
                'positions': self.positions
            })
            return True
        return False

    def update_portfolio(self, date, current_price):
        portfolio_value = self.capital + (self.positions * current_price)
        self.portfolio_values.append(portfolio_value)
        self.dates.append(date)
        self.holdings.append(self.positions)

    def get_trades_df(self):
        return pd.DataFrame(self.trades)

    def get_portfolio_df(self):
        return pd.DataFrame({
            'date': self.dates,
            'portfolio_value': self.portfolio_values,
            'holdings': self.holdings
        })


# ==================== TRADING STRATEGIES ====================

class TradingStrategies:

    @staticmethod
    def ml_threshold_strategy(predictions, actual_prices, threshold=0.02):
        signals = np.zeros(len(predictions))

        for i in range(1, len(predictions)):
            predicted_return = (predictions[i] - actual_prices[i-1]) / (actual_prices[i-1] + 1e-10)

            if predicted_return > threshold:
                signals[i] = 1
            elif predicted_return < -threshold:
                signals[i] = -1

        return signals

    @staticmethod
    def ml_momentum_strategy(predictions, actual_prices, window=5):
        signals = np.zeros(len(predictions))

        for i in range(window, len(predictions)):
            pred_momentum = np.mean(predictions[i-window:i])

            if predictions[i] > pred_momentum:
                signals[i] = 1
            else:
                signals[i] = -1

        return signals


# ==================== PERFORMANCE METRICS ====================

class PerformanceMetrics:

    @staticmethod
    def calculate_returns(portfolio_values):
        returns = pd.Series(portfolio_values).pct_change().dropna()
        return returns

    @staticmethod
    def total_return(portfolio_values):
        return ((portfolio_values[-1] / portfolio_values[0]) - 1) * 100

    @staticmethod
    def annualized_return(portfolio_values, days):
        total_ret = portfolio_values[-1] / portfolio_values[0]
        years = days / 365
        return (total_ret ** (1/years) - 1) * 100

    @staticmethod
    def sharpe_ratio(returns, risk_free_rate=0.05):
        excess_returns = returns - (risk_free_rate / 252)
        if excess_returns.std() == 0:
            return 0
        return np.sqrt(252) * (excess_returns.mean() / excess_returns.std())

    @staticmethod
    def sortino_ratio(returns, risk_free_rate=0.05):
        excess_returns = returns - (risk_free_rate / 252)
        downside_returns = returns[returns < 0]
        if len(downside_returns) == 0 or downside_returns.std() == 0:
            return 0
        return np.sqrt(252) * (excess_returns.mean() / downside_returns.std())

    @staticmethod
    def max_drawdown(portfolio_values):
        cummax = pd.Series(portfolio_values).cummax()
        drawdown = (pd.Series(portfolio_values) - cummax) / cummax
        return drawdown.min() * 100

    @staticmethod
    def win_rate(trades_df):
        if len(trades_df) == 0:
            return 0

        wins = 0
        total = 0
        for i in range(len(trades_df)-1):
            if trades_df.iloc[i]['type'] == 'BUY' and i+1 < len(trades_df):
                if trades_df.iloc[i+1]['type'] == 'SELL':
                    if trades_df.iloc[i+1].get('revenue', 0) > trades_df.iloc[i].get('cost', 0):
                        wins += 1
                    total += 1

        return (wins / total * 100) if total > 0 else 0

    @staticmethod
    def all_metrics(portfolio_values, trades_df, days):
        returns = PerformanceMetrics.calculate_returns(portfolio_values)

        metrics = {
            'Total Return (%)': PerformanceMetrics.total_return(portfolio_values),
            'Annualized Return (%)': PerformanceMetrics.annualized_return(portfolio_values, days),
            'Sharpe Ratio': PerformanceMetrics.sharpe_ratio(returns),
            'Sortino Ratio': PerformanceMetrics.sortino_ratio(returns),
            'Max Drawdown (%)': PerformanceMetrics.max_drawdown(portfolio_values),
            'Win Rate (%)': PerformanceMetrics.win_rate(trades_df),
            'Total Trades': len(trades_df),
            'Volatility (%)': returns.std() * np.sqrt(252) * 100 if len(returns) > 0 else 0
        }

        return metrics


# ==================== WALK-FORWARD BACKTEST ====================

def walk_forward_backtest(ticker='TCS.NS', period='5y',
                         train_window=500, test_window=60, step=30,
                         initial_capital=100000):
    """
    Walk-forward backtesting with model retraining
    """

    print(f"\n{'⏩'*40}")
    print("WALK-FORWARD BACKTESTING")
    print(f"{'⏩'*40}\n")

    # Download data
    print("📥 Downloading data...")
    data = yf.download(ticker, period=period, progress=False)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    # Add features
    print("🔧 Engineering features...")
    engineer = FeatureEngineer()
    data = engineer.add_all_features(data)
    data.dropna(inplace=True)

    print(f"✅ Prepared {len(data)} rows of data\n")

    # Feature columns
    feature_cols = ['Close', 'Volume', 'returns', 'volatility_10',
                    'rsi', 'macd', 'bb_position', 'volume_ratio',
                    'sma_20', 'ema_50', 'atr_percent']

    # Generate splits
    total_len = len(data)
    splits = []
    start = 0

    while start + train_window + test_window <= total_len:
        splits.append({
            'train_start': start,
            'train_end': start + train_window,
            'test_start': start + train_window,
            'test_end': start + train_window + test_window
        })
        start += step

    print(f"✅ Generated {len(splits)} walk-forward splits\n")

    # Initialize
    all_predictions = []
    all_actuals = []
    all_dates = []
    all_trades = []

    engine = BacktestEngine(initial_capital=initial_capital)
    position_held = False

    # Walk forward
    for i, split in enumerate(splits):
        print(f"Split {i+1}/{len(splits)}: Train[{split['train_start']}:{split['train_end']}] "
              f"Test[{split['test_start']}:{split['test_end']}]", end=' ')

        try:
            # Prepare data
            train_data = data.iloc[split['train_start']:split['train_end']]
            test_data = data.iloc[split['test_start']:split['test_end']]

            # Scale
            scaler = MinMaxScaler()
            train_scaled = scaler.fit_transform(train_data[feature_cols].values)
            test_scaled = scaler.transform(test_data[feature_cols].values)

            # Create sequences
            lookback = 50
            X_train, y_train = [], []
            for j in range(lookback, len(train_scaled)):
                X_train.append(train_scaled[j-lookback:j])
                y_train.append(train_scaled[j, 0])

            if len(X_train) == 0:
                print("❌ Not enough training data")
                continue

            X_train = np.array(X_train)
            y_train = np.array(y_train)

            X_test, y_test = [], []
            for j in range(lookback, len(test_scaled)):
                X_test.append(test_scaled[j-lookback:j])
                y_test.append(test_scaled[j, 0])

            if len(X_test) == 0:
                print("❌ Not enough test data")
                continue

            X_test = np.array(X_test)
            y_test = np.array(y_test)

            # Train model
            model = Sequential([
                LSTM(50, return_sequences=True, input_shape=(lookback, len(feature_cols))),
                Dropout(0.2),
                LSTM(32, return_sequences=False),
                Dropout(0.2),
                Dense(1)
            ])
            model.compile(optimizer='adam', loss='mse')

            early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
            model.fit(X_train, y_train, epochs=30, batch_size=32,
                     callbacks=[early_stop], verbose=0)

            # Predict
            preds_scaled = model.predict(X_test, verbose=0).flatten()

            # Inverse transform
            pred_full = np.zeros((len(preds_scaled), len(feature_cols)))
            pred_full[:, 0] = preds_scaled
            predictions = scaler.inverse_transform(pred_full)[:, 0]

            actual_full = np.zeros((len(y_test), len(feature_cols)))
            actual_full[:, 0] = y_test
            actuals = scaler.inverse_transform(actual_full)[:, 0]

            test_dates = test_data.index[lookback:]

            # Store
            all_predictions.extend(predictions)
            all_actuals.extend(actuals)
            all_dates.extend(test_dates)

            # Trade
            for j in range(len(predictions)):
                current_price = actuals[j]

                if j > 0:
                    predicted_return = (predictions[j] - actuals[j-1]) / actuals[j-1]

                    if predicted_return > 0.02 and not position_held:
                        if engine.buy(test_dates[j], current_price):
                            position_held = True

                    elif predicted_return < -0.02 and position_held:
                        if engine.sell(test_dates[j], current_price):
                            position_held = False

                engine.update_portfolio(test_dates[j], current_price)

            print("✅")

        except Exception as e:
            print(f"❌ Error: {str(e)}")
            continue

    # Close positions
    if position_held and len(all_actuals) > 0:
        engine.sell(all_dates[-1], all_actuals[-1])

    print("\n✅ Walk-forward backtest complete!\n")

    # Calculate metrics
    portfolio_df = engine.get_portfolio_df()
    trades_df = engine.get_trades_df()

    if len(all_dates) > 0:
        days = (all_dates[-1] - all_dates[0]).days
        metrics = PerformanceMetrics.all_metrics(
            portfolio_df['portfolio_value'].values,
            trades_df,
            days
        )

        # Buy and hold
        if len(all_actuals) > 0:
            buy_hold_return = ((all_actuals[-1] / all_actuals[0]) - 1) * 100
            metrics['Buy & Hold Return (%)'] = buy_hold_return
            metrics['Alpha (%)'] = metrics['Total Return (%)'] - buy_hold_return

        # Print results
        print("="*80)
        print("📊 WALK-FORWARD BACKTEST RESULTS")
        print("="*80)
        for key, value in metrics.items():
            print(f"{key:.<40} {value:>15.2f}")
        print("="*80)

        # Visualize
        plot_backtest_results({
            'ticker': ticker,
            'portfolio_df': portfolio_df,
            'trades_df': trades_df,
            'predictions': np.array(all_predictions),
            'actual_prices': np.array(all_actuals),
            'dates': all_dates,
            'metrics': metrics
        })

    return {
        'ticker': ticker,
        'predictions': np.array(all_predictions) if all_predictions else np.array([]),
        'actuals': np.array(all_actuals) if all_actuals else np.array([]),
        'dates': all_dates,
        'portfolio_df': portfolio_df,
        'trades_df': trades_df,
        'metrics': metrics if 'metrics' in locals() else {}
    }


# ==================== SIMPLE BACKTEST ====================

def simple_backtest(ticker, predictions, actual_prices, dates,
                   strategy='threshold', threshold=0.02, initial_capital=100000):
    """
    Simple backtest without retraining
    """

    print(f"\n{'='*80}")
    print(f"🎯 BACKTESTING: {ticker}")
    print(f"{'='*80}\n")

    # Initialize
    engine = BacktestEngine(initial_capital=initial_capital)
    strategies = TradingStrategies()

    # Generate signals
    if strategy == 'threshold':
        signals = strategies.ml_threshold_strategy(predictions, actual_prices, threshold)
    elif strategy == 'momentum':
        signals = strategies.ml_momentum_strategy(predictions, actual_prices, window=5)
    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    # Simulate trading
    position_held = False

    for i in range(len(signals)):
        current_price = actual_prices[i]
        signal = signals[i]

        if signal == 1 and not position_held:
            if engine.buy(dates[i], current_price):
                position_held = True

        elif signal == -1 and position_held:
            if engine.sell(dates[i], current_price):
                position_held = False

        engine.update_portfolio(dates[i], current_price)

    # Close positions
    if position_held:
        engine.sell(dates[-1], actual_prices[-1])

    # Get results
    trades_df = engine.get_trades_df()
    portfolio_df = engine.get_portfolio_df()

    # Calculate metrics
    days = (dates[-1] - dates[0]).days
    metrics = PerformanceMetrics.all_metrics(
        portfolio_df['portfolio_value'].values,
        trades_df,
        days
    )

    # Buy and hold
    buy_hold_return = ((actual_prices[-1] / actual_prices[0]) - 1) * 100
    metrics['Buy & Hold Return (%)'] = buy_hold_return
    metrics['Alpha (%)'] = metrics['Total Return (%)'] - buy_hold_return

    # Print
    print("📊 PERFORMANCE METRICS")
    print("="*80)
    for key, value in metrics.items():
        print(f"{key:.<40} {value:>15.2f}")
    print("="*80)

    result = {
        'ticker': ticker,
        'portfolio_df': portfolio_df,
        'trades_df': trades_df,
        'predictions': predictions,
        'actual_prices': actual_prices,
        'dates': dates,
        'metrics': metrics
    }

    plot_backtest_results(result)

    return result


# ==================== VISUALIZATION ====================

def plot_backtest_results(results):
    """Create backtest visualization"""

    portfolio_df = results['portfolio_df']
    trades_df = results['trades_df']
    dates = results['dates']
    actual_prices = results['actual_prices']

    # Create figure
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Portfolio Value',
            'Stock Price with Signals',
            'Cumulative Returns',
            'Drawdown'
        ),
        vertical_spacing=0.15,
        horizontal_spacing=0.12
    )

    # 1. Portfolio value
    initial = portfolio_df['portfolio_value'].iloc[0]
    buy_hold = initial * (actual_prices / actual_prices[0])

    fig.add_trace(
        go.Scatter(x=portfolio_df['date'], y=portfolio_df['portfolio_value'],
                  name='Strategy', line=dict(color='blue', width=2)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=dates, y=buy_hold,
                  name='Buy & Hold', line=dict(color='gray', width=2, dash='dash')),
        row=1, col=1
    )

    # 2. Price with signals
    fig.add_trace(
        go.Scatter(x=dates, y=actual_prices,
                  name='Price', line=dict(color='black', width=1)),
        row=1, col=2
    )

    if len(trades_df) > 0:
        buy_trades = trades_df[trades_df['type'] == 'BUY']
        sell_trades = trades_df[trades_df['type'] == 'SELL']

        if len(buy_trades) > 0:
            fig.add_trace(
                go.Scatter(x=buy_trades['date'], y=buy_trades['price'],
                          mode='markers', name='Buy',
                          marker=dict(symbol='triangle-up', size=10, color='green')),
                row=1, col=2
            )

        if len(sell_trades) > 0:
            fig.add_trace(
                go.Scatter(x=sell_trades['date'], y=sell_trades['price'],
                          mode='markers', name='Sell',
                          marker=dict(symbol='triangle-down', size=10, color='red')),
                row=1, col=2
            )

    # 3. Cumulative returns
    strategy_returns = (portfolio_df['portfolio_value'] / initial - 1) * 100
    buy_hold_returns = (buy_hold / initial - 1) * 100

    fig.add_trace(
        go.Scatter(x=portfolio_df['date'], y=strategy_returns,
                  name='Strategy Returns', line=dict(color='blue', width=2)),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(x=dates, y=buy_hold_returns,
                  name='B&H Returns', line=dict(color='gray', width=2, dash='dash')),
        row=2, col=1
    )

    # 4. Drawdown
    cummax = portfolio_df['portfolio_value'].cummax()
    drawdown = (portfolio_df['portfolio_value'] - cummax) / cummax * 100

    fig.add_trace(
        go.Scatter(x=portfolio_df['date'], y=drawdown, fill='tozeroy',
                  name='Drawdown', line=dict(color='red', width=1)),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        height=800,
        title_text=f"{results.get('ticker', 'Stock')} - Backtest Results",
        title_x=0.5,
        showlegend=True,
        template='plotly_white'
    )

    fig.update_yaxes(title_text="Value (₹)", row=1, col=1)
    fig.update_yaxes(title_text="Price (₹)", row=1, col=2)
    fig.update_yaxes(title_text="Return (%)", row=2, col=1)
    fig.update_yaxes(title_text="Drawdown (%)", row=2, col=2)

    fig.show()


# ==================== USAGE ====================

print("""
╔═══════════════════════════════════════════════════════════════════════╗
║          BACKTESTING SYSTEM LOADED SUCCESSFULLY!                      ║
╚═══════════════════════════════════════════════════════════════════════╝

""")


╔═══════════════════════════════════════════════════════════════════════╗
║          BACKTESTING SYSTEM LOADED SUCCESSFULLY!                      ║
╚═══════════════════════════════════════════════════════════════════════╝

 QUICK START:



 Simple Backtest (After ML predictions):
   results = simple_backtest(
       ticker='TCS.NS',
       predictions=your_predictions,
       actual_prices=actual_prices,
       dates=dates,
       strategy='threshold',
       threshold=0.02
   

═══════════════════════════════════════════════════════════════════════



In [13]:

results = walk_forward_backtest(
    ticker='TCS.NS',
    period='5y',
    train_window=500,  # Train on 500 days
    test_window=60,    # Test on 60 days
    step=30,           # Retrain every 30 days
    initial_capital=100000
)


⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩
WALK-FORWARD BACKTESTING
⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩

📥 Downloading data...
🔧 Engineering features...
✅ Prepared 1038 rows of data

✅ Generated 16 walk-forward splits

Split 1/16: Train[0:500] Test[500:560] ✅
Split 2/16: Train[30:530] Test[530:590] ✅
Split 3/16: Train[60:560] Test[560:620] ✅
Split 4/16: Train[90:590] Test[590:650] 

✅
Split 5/16: Train[120:620] Test[620:680] 

✅
Split 6/16: Train[150:650] Test[650:710] ✅
Split 7/16: Train[180:680] Test[680:740] ✅
Split 8/16: Train[210:710] Test[710:770] ✅
Split 9/16: Train[240:740] Test[740:800] ✅
Split 10/16: Train[270:770] Test[770:830] ✅
Split 11/16: Train[300:800] Test[800:860] ✅
Split 12/16: Train[330:830] Test[830:890] ✅
Split 13/16: Train[360:860] Test[860:920] ✅
Split 14/16: Train[390:890] Test[890:950] ✅
Split 15/16: Train[420:920] Test[920:980] ✅
Split 16/16: Train[450:950] Test[950:1010] ✅

✅ Walk-forward backtest complete!

📊 WALK-FORWARD BACKTEST RESULTS
Total Return (%)........................           -6.41
Annualized Return (%)...................           -3.49
Sharpe Ratio............................           -0.55
Sortino Ratio...........................           -0.41
Max Drawdown (%)........................          -20.98
Win Rate (%)............................           66.67
Total Trades............................            6.00
Volatility (%)..........................          

In [14]:
results = walk_forward_backtest('TCS.NS', period='5y')


⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩
WALK-FORWARD BACKTESTING
⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩

📥 Downloading data...
🔧 Engineering features...
✅ Prepared 1038 rows of data

✅ Generated 16 walk-forward splits

Split 1/16: Train[0:500] Test[500:560] ✅
Split 2/16: Train[30:530] Test[530:590] ✅
Split 3/16: Train[60:560] Test[560:620] ✅
Split 4/16: Train[90:590] Test[590:650] ✅
Split 5/16: Train[120:620] Test[620:680] ✅
Split 6/16: Train[150:650] Test[650:710] ✅
Split 7/16: Train[180:680] Test[680:740] ✅
Split 8/16: Train[210:710] Test[710:770] ✅
Split 9/16: Train[240:740] Test[740:800] ✅
Split 10/16: Train[270:770] Test[770:830] ✅
Split 11/16: Train[300:800] Test[800:860] ✅
Split 12/16: Train[330:830] Test[830:890] ✅
Split 13/16: Train[360:860] Test[860:920] ✅
Split 14/16: Train[390:890] Test[890:950] ✅
Split 15/16: Train[420:920] Test[920:980] ✅
Split 16/16: Train[450:950] Test[950:1010] ✅

✅ Walk-forward backtest complete!

📊 WALK-FORWARD BACKTEST RESULTS
Total Return (%)..